## To run this file, install Python packages: 
- requests
- tabulate

In [89]:
import json
import requests
from random import randrange
from tabulate import tabulate

# Year range
start_year = 2010
end_year = 2020
year = randrange(start_year, end_year)
game_season = str(year) + "-" + str((year + 1) % 100).rjust(2, "0")

# Game type
game_types = ["Regular Season", "Playoffs"]
game_type = game_types[randrange(len(game_types))]

# Game data
players_url = "https://api.pbpstats.com/get-games/nba"
games_params = {
    "Season": game_season,
    "SeasonType": game_type
}
response = requests.get(players_url, params=games_params)
response_json = response.json()["results"]

# Random a game
game_id= randrange(len(response_json))
game_overview = response_json[game_id]
print(json.dumps(game_overview, indent=2))

# Get game stats from the overview
print("Extracting game stats for game:", game_overview["GameId"], "...")
players_url = "https://api.pbpstats.com/get-game-stats"
games_params = {
    "GameId": game_overview["GameId"],
    "Type": "Player" # options are Player, Lineup and LineupOpponent
}
response = requests.get(players_url, params=games_params)
game_stats = response.json()['stats']
print("Done")

{
  "GameId": "0041500125",
  "Date": "2016-04-27",
  "HomeTeamId": "1610612748",
  "AwayTeamId": "1610612766",
  "HomePoints": 88,
  "AwayPoints": 90,
  "HomePossessions": 96,
  "AwayPossessions": 98,
  "HomeTeamAbbreviation": "MIA",
  "AwayTeamAbbreviation": "CHA"
}
Extracting game stats for game: 0041500125 ...
Done


### Some util functions:

In [90]:
# Stats to get
key_general = ["GamesPlayed"]
key_attack_stats = ["Fg2Pct", "Fg3Pct", "FtPoints", "FTA", "Turnovers", "TotalPoss"]
key_defend_stats = ["Steals", "Blocks", "Fouls"]
key_all_stats = key_general + key_attack_stats + key_defend_stats

# Uitil functions
def get_team_players(game_stats, team_type):
    team = []
    if team_type == "Away":
        stats = game_stats["Away"]["FullGame"]
    elif team_type == "Home":
        stats = game_stats["Home"]["FullGame"]

    for entity in stats:
        if entity["Name"] != "Team":
            team.append([entity["Name"], entity["Minutes"]])
    team.sort(key=lambda lst: lst[1], reverse=True)
    return team


def get_player_ids(team, team_type):
    if team_type == "Away":
        team_id = game_overview["AwayTeamId"]
    elif team_type == "Home":
        team_id = game_overview["HomeTeamId"]

    players_url = "https://api.pbpstats.com/get-team-players-for-season"
    players_params = {
        "Season": game_season,
        "SeasonType": game_type,
        "TeamId": team_id
    }
    players_response = requests.get(players_url, params=players_params)
    players = players_response.json()["players"]

    # "team" is list of list, each sub list starts with player name
    for player in team:
        for id, name in players.items():
            if name == player[0]:
                player.append(id)

def get_player_stats_by_id(id):
    url = "https://api.pbpstats.com/get-all-season-stats/nba"
    params = {
        "EntityType": "Player",
        "EntityId": id
    }
    player_stats = requests.get(url, params=params).json()

    for json_object in player_stats["results"][game_type]:
        for key, value in json_object.items():
            if key == "Season" and value == game_season:
                return json_object


def get_team_stats(team):
    for player in team:
        all_stats = get_player_stats_by_id(player[2])
        for key in key_all_stats:
            if key in all_stats:
                player.append(all_stats[key])
            else:
                player.append(None)
        # Add player position
        # player.append(get_player_position(player[2]))

def get_player_position(player_id):
    from nba_api.stats.endpoints import commonplayerinfo
    player_info = commonplayerinfo.CommonPlayerInfo(player_id)
    player_info = player_info.get_normalized_dict()
    # Map the position to 5 short terms: PG, SG, SF, PF, C
    position_map = {
        "" : "PG", # Default to "PG"
        "Guard": "PG",
        "Point-Guard": "PG",
        "Shooting-Guard": "SG",
        "Forward-Guard": "SG",
        "Forward": "SF",
        "Guard-Forward": "SF",
        "Small-Forward": "SF",
        "Power-Forward": "PF",
        "Center-Forward": "PF",
        "Center": "C",
        "Forward-Center": "C",
    }
    return position_map[player_info["CommonPlayerInfo"][0]["POSITION"]]

# Print messages
col_names = ["Name", "Minutes", "ID"] + key_all_stats
def print_table(team):
    print(tabulate(team, headers=col_names, tablefmt="grid"))

## Export origin game results

In [102]:
# Export the game result to the CSV file
import csv

with open("data/game_origin_results.csv", "w", newline="") as file:
    writer = csv.writer(file)
    # check if the file is empty
    if file.tell() == 0:
        print("Empty file, Writing the header to the file")
        writer.writerow(["game_id", 
                        "home_id", 
                        "away_id", 
                        "home_abbr", 
                        "away_abbr", 
                        "home_points", 
                        "away_points"])


# Check if the game is already in the file by the game ID
with open("data/game_origin_results.csv", "r") as file:
    reader = csv.reader(file)
    for row in reader:
        if row[0] == game_overview["GameId"]:
            print("The game is already in the file")
            break
    else:
        print("Writing the game to the file")
        with open("data/game_origin_results.csv", "a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow([game_overview["GameId"],
                            game_overview["HomeTeamId"],
                            game_overview["AwayTeamId"],
                            game_overview["HomeTeamAbbreviation"],
                            game_overview["AwayTeamAbbreviation"],
                            game_overview["HomePoints"],
                            game_overview["AwayPoints"]])
                


Writing the game to the file


## Away team stats:

In [92]:
# # Away team stats
# print("Away team players:")
# away_team_players_data = {}
# away_team = get_team_players(game_stats, "Away")
# get_player_ids(away_team, "Away")
# get_team_stats(away_team)
# print_table(away_team)

## Home team stats:

In [93]:
# # Home team stats
# print("Home team players:")
# home_team = get_team_players(game_stats, "Home")
# get_player_ids(home_team, "Home")
# get_team_stats(home_team)
# print_table(home_team)

## Aggregate and export the both teams players key stats to CSV

In [94]:
import csv

# Get the lineup players key stats
def export_lineup(game_stats, is_home):
    team_type = "Home" if is_home else "Away"
    team_stats = get_team_players(game_stats, team_type)
    get_player_ids(team_stats, team_type)
    get_team_stats(team_stats)
    col_names = ["Name", "Minutes", "ID"] + key_all_stats
    team_stats = [{col_names[i]: player[i] for i in range(len(col_names))} for player in team_stats]
    
    for player in team_stats:
        player["Position"] = get_player_position(player["ID"])
        # Calculate the PassRatio
        # If Turnovers or TotalPoss is None, then use the average value of the team
        if player["Turnovers"] is None or player["TotalPoss"] is None:
            team_turnovers = [player["Turnovers"] for player in team_stats if player["Turnovers"] is not None]
            team_total_poss = [player["TotalPoss"] for player in team_stats if player["TotalPoss"] is not None]
            player["Turnovers"] = sum(team_turnovers) / len(team_turnovers)
            player["TotalPoss"] = sum(team_total_poss) / len(team_total_poss)
        player["PassRatio"] = 1 - player["Turnovers"] / player["TotalPoss"]

    print(tabulate(team_stats[:], headers="keys", tablefmt="grid"))

    positions = ["PG", "SG", "SF", "PF", "C"]
    lineup = {}
    lineup_ids = []
    for position in positions:
        lineup[position] = None
    for player in team_stats:
        if lineup[player["Position"]] is None and player["Position"] in positions:
            lineup[player["Position"]] = player
            lineup_ids.append(player["ID"])

    for position in lineup:
        if lineup[position] is None:
            for player in team_stats:
                if player['ID'] not in lineup_ids:
                    lineup[position] = player
                    lineup_ids.append(player["ID"])
                    break

    print(f"The lineup of the {team_type} team:")
    for position in lineup:
        print(position + ":", lineup[position])

    # Export the lineup to a CSV file
    # The columns include position, PassRatio, Fg2Pct, ID, name
    LINEUP_FILE = f"data/lineup_{team_type.lower()}_{game_overview['GameId']}.csv"
    with open(LINEUP_FILE, mode="w") as file:
        writer = csv.writer(file)
        writer.writerow(["Position", "PassRatio", "Fg2Pct", "ID", "Name"])
        for position in lineup:
            player = lineup[position]
            writer.writerow([position, player["PassRatio"], player["Fg2Pct"], player["ID"], player["Name"]])
    
    return lineup

export_lineup(game_stats, is_home=True)
export_lineup(game_stats, is_home=False)



+------------------+-----------+---------+---------------+----------+----------+------------+-------+-------------+-------------+----------+----------+---------+------------+-------------+
| Name             | Minutes   |      ID |   GamesPlayed |   Fg2Pct |   Fg3Pct |   FtPoints |   FTA |   Turnovers |   TotalPoss |   Steals |   Blocks |   Fouls | Position   |   PassRatio |
+==================+===========+=========+===============+==========+==========+============+=======+=============+=============+==========+==========+=========+============+=============+
| Joe Johnson      | 36:27     |    2207 |            14 | 0.491071 | 0.282609 |         21 |    24 |          18 |        1846 |        8 |        3 |      27 | SF         |    0.990249 |
+------------------+-----------+---------+---------------+----------+----------+------------+-------+-------------+-------------+----------+----------+---------+------------+-------------+
| Luol Deng        | 36:22     |    2736 |            1

{'PG': {'Name': 'Kemba Walker',
  'Minutes': '35:01',
  'ID': '202689',
  'GamesPlayed': 7,
  'Fg2Pct': 0.38181818181818183,
  'Fg3Pct': 0.32558139534883723,
  'FtPoints': 33,
  'FTA': 35,
  'Turnovers': 12,
  'TotalPoss': 978,
  'Steals': 9,
  'Blocks': 4,
  'Fouls': 10,
  'Position': 'PG',
  'PassRatio': 0.9877300613496932},
 'SG': {'Name': 'Jeremy Lin',
  'Minutes': '34:33',
  'ID': '202391',
  'GamesPlayed': 7,
  'Fg2Pct': 0.46938775510204084,
  'Fg3Pct': 0.21428571428571427,
  'FtPoints': 32,
  'FTA': 39,
  'Turnovers': 11,
  'TotalPoss': 720,
  'Steals': 5,
  'Blocks': None,
  'Fouls': 21,
  'Position': 'PG',
  'PassRatio': 0.9847222222222223},
 'SF': {'Name': 'Marvin Williams',
  'Minutes': '30:23',
  'ID': '101107',
  'GamesPlayed': 7,
  'Fg2Pct': 0.23529411764705882,
  'Fg3Pct': 0.35294117647058826,
  'FtPoints': 2,
  'FTA': 4,
  'Turnovers': 2,
  'TotalPoss': 860,
  'Steals': 6,
  'Blocks': 3,
  'Fouls': 11,
  'Position': 'SF',
  'PassRatio': 0.9976744186046511},
 'PF': {'Nam

## Generate new PAT PCSP files for the game


In [95]:
# Read the lineup from the CSV file
def read_lineup(team_type):
    LINEUP_FILE = f"data/lineup_{team_type.lower()}_{game_overview['GameId']}.csv"
    with open(LINEUP_FILE, mode="r") as file:
        reader = csv.reader(file)
        lineup = []
        for i, row in enumerate(reader):
            if i == 0:
                continue
            lineup.append(row)
    return lineup

# Generate the PCSP lineup stats for the game base on the template file "game_template.pcsp"
# Replace the first lines of the below code with the lineup stats
def generate_pcsp_lineup(team_type):
    lineup = read_lineup(team_type)
    positions = ["PG", "SG", "SF", "PF", "C"]
    stats = ""
    for position in positions:
        for player in lineup:
            if player[0] == position:
                stats += f"#define player{position}Pass {int(float(player[1]) * 100)};\n"
                stats += f"#define player{position}Score {int(float(player[2]) * 100)};\n"

    PCSP_TEMPLATE = "game_template.pcsp"
    PCSP_FILE = f"game_{game_overview['GameId']}_{team_type.lower()}.pcsp"
    with open(PCSP_TEMPLATE, mode="r") as file:
        template = file.read()
    with open(PCSP_FILE, mode="w") as file:
        # Replace the first 10 lines of the template with the lineup stats
        # Remove the first 10 lines of the template
        template = template.split("\n")[10:]
        file.write(stats)
        file.write("\n")
        file.write("\n".join(template))


generate_pcsp_lineup("Home")
generate_pcsp_lineup("Away")